<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#概要" data-toc-modified-id="概要-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>概要</a></span></li><li><span><a href="#機械学習における前処理" data-toc-modified-id="機械学習における前処理-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>機械学習における前処理</a></span></li><li><span><a href="#階層カラムを持つデータフレームの操作方法" data-toc-modified-id="階層カラムを持つデータフレームの操作方法-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>階層カラムを持つデータフレームの操作方法</a></span></li><li><span><a href="#Permutaion-Importance" data-toc-modified-id="Permutaion-Importance-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Permutaion Importance</a></span><ul class="toc-item"><li><span><a href="#Eli5-パッケージを使う" data-toc-modified-id="Eli5-パッケージを使う-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Eli5 パッケージを使う</a></span></li></ul></li><li><span><a href="#良い教材サイト" data-toc-modified-id="良い教材サイト-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>良い教材サイト</a></span></li></ul></div>

# 概要
学習メモである。2019-02-04以降の内容である。


# 機械学習における前処理

おさらいであるが、良いエントリを見つけた。

Machine Learning(ML) — Data Preprocessing – Data Driven Investor – Medium
https://medium.com/datadriveninvestor/machine-learning-ml-data-preprocessing-5b346766fc48


機械学習における前処理は、主には次の５つに分けられる。 
* データクリーニング
    1. 欠損値を補完する
    2. ノイズを含むデータを平滑化する
    3. 異常値を識別する、または除去する
    4. その他不整合を解決する
* データ統合
    複数のデータベース、テーブル、ファイルを利用して、データを結合しする。
* データ変換
    正規化や集約処理をする
* データ削減
    データ量を減らしつつも、同じか類似した分析結果を得られるようにする。
* データ離散化
    数値をone-hot形式に変換するなど

> Data cleaning: fill in missing values, smooth noisy data, identify or remove outliers, and resolve inconsistencies.  
Data integration: using multiple databases, data cubes, or files.  
Data transformation: normalization and aggregation.  
Data reduction: reducing the volume but producing the same or similar analytical results.  
Data discretization: part of data reduction, replacing numerical attributes with nominal ones.  


# 階層カラムを持つデータフレームの操作方法
カラムが階層構造を持つ場合、タプルを用いて各階層のカラムを指定すれば良い。ワイルドカード指定をしたい場合は、カラム指定をスキップすれば良い。  

```python
# カラムは３階層構造を持っている。第一階層では "create_time" と "receive_time" を指定し、
# 第二階層では "LMAX_LD" を指定。
df.loc[df.index[0:100],(("create_time","receive_time"),"LMAX_LD")]
```

# Permutaion Importance
構築したモデルにおいて、どの特徴量が重要なのかを、各特徴量の並び順をランダムにシャッフルしながら求めたもの。  
モデルを構築しなおすのではなく、値を入れ替えたもので予測させるので、計算負荷に考慮した手法である。  

[Python: 特徴量の重要度を Permutation Importance で計測する - CUBE SUGAR CONTAINER](https://blog.amedama.jp/entry/permutation-importance)  
おなじみのiris dataset をランダムフォレストで学習させ、構築したモデルにおける各特徴量の影響度を Permutation Importance として計算。  
ランダムに並べ替えると性能悪化が大きい特徴量は ```Petal length```, ```Petal width``` である。  
性能悪化のばらつきの幅も大きい。  
一方、影響が少ないのは```Sepal length```, ```Sepal width``` で、性能悪化はほぼゼロ。ばらつきも小さい。   
![Permutation Importance](https://cdn-ak.f.st-hatena.com/images/fotolife/m/momijiame/20181110/20181110230620.png)


Permutation Importance の話題からそれるが、以下のように yield を使うと、イテレーション処理で呼び出すジェネレータを作ることが出来る。  
```python
def permuted(df):
    """特定のカラムをシャッフルしたデータフレームを返す"""
    for column_name in df.columns:
        permuted_df = df.copy()
        permuted_df[column_name] = np.random.permutation(permuted_df[column_name])
        yield column_name, permuted_df
        
# このようなイテレーション処理に使える。
for column_name, permuted_X_test in permuted(X_test):
    # 処理
```

## Eli5 パッケージを使う

Permutation Importance を計算するための```Eli5```というパッケージがある。  
以下のエントリで使い方を紹介している。  
[機械学習モデルにおける特徴量の重要度評価Permutation Importanceとeli5について 実装編 | 分析トレイン](https://bunseki-train.com/permutaion_importance_and_eli5_2/)

こちらも、お馴染みのワインデータセットをランダムフォレストで学習し、Eli5パッケージを使って Permutation Importance を求めている。  
(このような表形式でレンダリングする機能も提供されている。)
![Permutation Importance @ Wine](https://bunseki-train.com/wp/wp-content/uploads/2018/09/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2018-09-22-00.27.24-300x200.png)

そして、影響度上位２つの特徴量を散布図にすると、以下のような図が得られた。  
![Scatter Plot@Wine](https://bunseki-train.com/wp/wp-content/uploads/2018/09/output_4_0.png)

他にも、個々のサンプルにおける特徴量の重要度を計算するAPIも提供されている。  
```python
# コード例
eli5.show_prediction(rf_clf, doc=val_X.iloc[0], feature_names= val_X.columns.tolist(), show_feature_values=True)
```
![Permutation Importance@each sample](https://bunseki-train.com/wp/wp-content/uploads/2018/09/%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%B3%E3%82%B7%E3%83%A7%E3%83%83%E3%83%88-2018-09-22-18.57.46-1024x232.png)

# 良い教材サイト

以下のサイトのRSSは、定点観測しておくべきだろう。
https://towardsdatascience.com/